In [1]:
import tensorflow as tf
import numpy as np
import cv2
from object_detection.utils import visualization_utils as vis_util

def load_frozen_model(model_path):
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.compat.v1.GraphDef()
        with tf.io.gfile.GFile(model_path, 'rb') as f:
            serialized_graph = f.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

    return detection_graph

def detect(detection_graph, category_index):
    cap = cv2.VideoCapture(0) 

    with detection_graph.as_default():
        with tf.compat.v1.Session() as sess:
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

            while True:
                ret, frame = cap.read() 
                image_np = np.array(frame)
                expanded_dims = np.expand_dims(image_np, axis=0)

                (boxes, scores, classes, num) = sess.run(
                    [detection_boxes, detection_scores, detection_classes, num_detections],
                    feed_dict={image_tensor: expanded_dims}
                )

                vis_util.visualize_boxes_and_labels_on_image_array(
                    image_np,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores>0.615),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=5
                )

                image_np = cv2.resize(image_np,(840,660))
                cv2.imshow('Litter Detection', image_np) 

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

    cap.release()
    cv2.destroyAllWindows()

model_path = 'litter_detection_ssd.pb'

detection_graph = load_frozen_model(model_path)

category_index = {1: {'id': 1, 'name': 'Aluminium foil'}, 2: {'id': 2, 'name': 'Battery'}, 3: {'id': 3, 'name': 'Aluminium blister pack'}, 4: {'id': 4, 'name': 'Carded blister pack'}, 5: {'id': 5, 'name': 'Other plastic bottle'}, 6: {'id': 6, 'name': 'Clear plastic bottle'}, 7: {'id': 7, 'name': 'Glass bottle'}, 8: {'id': 8, 'name': 'Plastic bottle cap'}, 9: {'id': 9, 'name': 'Metal bottle cap'}, 10: {'id': 10, 'name': 'Broken glass'}, 11: {'id': 11, 'name': 'Food Can'}, 12: {'id': 12, 'name': 'Aerosol'}, 13: {'id': 13, 'name': 'Drink can'}, 14: {'id': 14, 'name': 'Toilet tube'}, 15: {'id': 15, 'name': 'Other carton'}, 16: {'id': 16, 'name': 'Egg carton'}, 17: {'id': 17, 'name': 'Drink carton'}, 18: {'id': 18, 'name': 'Corrugated carton'}, 19: {'id': 19, 'name': 'Meal carton'}, 20: {'id': 20, 'name': 'Pizza box'}, 21: {'id': 21, 'name': 'Paper cup'}, 22: {'id': 22, 'name': 'Disposable plastic cup'}, 23: {'id': 23, 'name': 'Foam cup'}, 24: {'id': 24, 'name': 'Glass cup'}, 25: {'id': 25, 'name': 'Other plastic cup'}, 26: {'id': 26, 'name': 'Food waste'}, 27: {'id': 27, 'name': 'Glass jar'}, 28: {'id': 28, 'name': 'Plastic lid'}, 29: {'id': 29, 'name': 'Metal lid'}, 30: {'id': 30, 'name': 'Other plastic'}, 31: {'id': 31, 'name': 'Magazine paper'}, 32: {'id': 32, 'name': 'Tissues'}, 33: {'id': 33, 'name': 'Wrapping paper'}, 34: {'id': 34, 'name': 'Normal paper'}, 35: {'id': 35, 'name': 'Paper bag'}, 36: {'id': 36, 'name': 'Plastified paper bag'}, 37: {'id': 37, 'name': 'Plastic film'}, 38: {'id': 38, 'name': 'Six pack rings'}, 39: {'id': 39, 'name': 'Garbage bag'}, 40: {'id': 40, 'name': 'Other plastic wrapper'}, 41: {'id': 41, 'name': 'Single-use carrier bag'}, 42: {'id': 42, 'name': 'Polypropylene bag'}, 43: {'id': 43, 'name': 'Crisp packet'}, 44: {'id': 44, 'name': 'Spread tub'}, 45: {'id': 45, 'name': 'Tupperware'}, 46: {'id': 46, 'name': 'Disposable food container'}, 47: {'id': 47, 'name': 'Foam food container'}, 48: {'id': 48, 'name': 'Other plastic container'}, 49: {'id': 49, 'name': 'Plastic glooves'}, 50: {'id': 50, 'name': 'Plastic utensils'}, 51: {'id': 51, 'name': 'Pop tab'}, 52: {'id': 52, 'name': 'Rope & strings'}, 53: {'id': 53, 'name': 'Scrap metal'}, 54: {'id': 54, 'name': 'Shoe'}, 55: {'id': 55, 'name': 'Squeezable tube'}, 56: {'id': 56, 'name': 'Plastic straw'}, 57: {'id': 57, 'name': 'Paper straw'}, 58: {'id': 58, 'name': 'Styrofoam piece'}, 59: {'id': 59, 'name': 'Unlabeled litter'}, 60: {'id': 60, 'name': 'Cigarette'}}

for category_id in range(1, 61):
    category_index[category_id]['name'] = 'Litter'

detect(detection_graph, category_index)


2023-09-19 13:51:37.644462: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-19 13:51:37.759857: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-19 13:51:37.760482: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-19 13:51:38.740729: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-19 13:51:41.194249: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
